1. Triplet Input Format : user_id , movie_id , pref_score , rating
2. Modality1 : Audio
   Modality2 : Video
3. Test - Train Split is Defined Within code So, Code Doesnt Requires Seprate File
4. Its is for Classification based Problem
5. ALL Embeddings Dimension is same
6. Here Batch Size for val and Test is compelete set

Changes Req

1. Path of Input files
2. Pah to save checkpoint
3. Path of Data File

## Importing Modules


In [ ]:
import torch
import torch.nn as nn
import pickle5 as pickle
import pandas as pd
import os
import torch.optim as optim
import torch
from tqdm import tqdm
from torchmetrics.functional.classification import multilabel_accuracy
import numpy as np
from sklearn.metrics import classification_report

from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

### Support

In [ ]:
def load_pkl(filename):
    filename= filename
    data= None
    with open(filename, "rb") as handle:
        data= pickle.load(handle)
        handle.close()
    return data

def store_pkl(object, filename):
    filename= path_of(filename)
    with open(filename, "wb") as handle:
        pickle.dump(object, handle, protocol=pickle.HIGHEST_PROTOCOL)
        handle.close()


def checkpoint(model, filename):
    torch.save(model.state_dict(), "/home/prabir_prj22/Pulkit/MFVCD_7K/Save/"+filename) #Change

def resume(filename):
    model.load_state_dict(torch.load("/home/prabir_prj22/Pulkit/MFVCD_7K/Save/"+filename)) #Change

### DataSet Loader


In [ ]:
from ast import literal_eval
class customDataset(torch.utils.data.Dataset):
    def __init__(self, data_file ,modality1 , user_modality1 , modality2 , user_modality2):

        self.triplet = data_file

        self.UserModality1_representation = load_pkl(user_modality1)
        self.modality1_representation = load_pkl(modality1)

        self.UserModality2_representation = load_pkl(user_modality2)
        self.modality2_representation = load_pkl(modality2)


    def __len__(self):
        return len(self.triplet)

    def __getitem__(self, idx):
        user_id , movie_id , pref_score , rating  = self.triplet[idx]

        #Modality1 Embedding
        UserModality1_embed = torch.FloatTensor(self.UserModality1_representation [user_id]).unsqueeze(0)
        modality1_embed = pref_score * 100 * torch.FloatTensor(self.modality1_representation[movie_id]).unsqueeze(0)

        #Modality2 Embedding
        UserModality2_embed = torch.FloatTensor(self.UserModality2_representation [user_id]).unsqueeze(0)
        modality2_embed = pref_score * 100 * torch.FloatTensor(self.modality2_representation[movie_id]).unsqueeze(0)



        #Label
        y_label = torch.FloatTensor(self.onehot(rating[0]))

        return movie_id, modality1_embed , UserModality1_embed , modality2_embed , UserModality2_embed , y_label

    def onehot(self,rate):
        if rate == '-1':
            return([1.,0.,0.])

        elif rate == '1':
            return([0.,0.,1.])

        elif rate == '0':
            return([0.,1.,0.])


### Report

In [ ]:
def cal_accuracy(y_test, y_pred):
    with open("/home/prabir_prj22/Pulkit/User_Modality_Model/Results/xyz.txt", "a") as h: #Change
        print("\n")
        # print("Confusion Matrix:\n ",confusion_matrix(y_test, y_pred))
        print("Confusion Matrix:\n ",confusion_matrix(y_test, y_pred), file=h)

        print ("Accuracy :\n ",accuracy_score(y_test,y_pred)*100, file=h)

        print("Report : \n", classification_report(y_test, y_pred), file=h)
        # print("Report : \n", classification_report(y_test, y_pred))
        precision = precision_score(y_test, y_pred, average = 'micro')
        print('Precision: %f' % precision, file=h)
        # recall: tp / (tp + fn)
        recall = recall_score(y_test, y_pred, average = 'micro')
        print('Recall: %f' % recall, file=h)
        # f1: 2 tp / (2 tp + fp + fn)
        f1 = f1_score(y_test, y_pred, average = 'micro')
        print('F1 score: %f' % f1, file=h)

### Loading Files

In [ ]:
#Change

#Movie Embeddings

modality1 = "/home/prabir_prj22/Pulkit/User_Modality_Model/Dataset/w2v/wave2vec_movie_audio.pkl"
modality2 = "/home/prabir_prj22/Pulkit/User_Modality_Model/Dataset/scenedetection_object_detection_video_embeddings/scenedetection_object_detection_video_embeddings.pkl"

#User Embeddings
user_modality1 = "/home/prabir_prj22/Pulkit/User_Modality_Model/Dataset/w2v/user_emb_all_Score.pkl"
user_modality2 = "/home/prabir_prj22/Pulkit/User_Modality_Model/Dataset/scenedetection_object_detection_video_embeddings/user_emb_all_Score.pkl"

#Data Triplet
data_file_final = load_pkl("/home/prabir_prj22/Pulkit/User_Modality_Model/Dataset/Others/data_file_final.pkl") #Change

print(len(data_file_final))

## Model

In [ ]:
import torch.nn.functional as F

class CrossAttention(nn.Module):
    def __init__(self, input_dim:int ,
                 num_heads:int ,
                 dropout:int
                ):
        super(CrossAttention, self).__init__()

        # Multi-head attention layer
        self.attention = nn.MultiheadAttention(input_dim, num_heads)
        self.norm = nn.LayerNorm(input_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input1, input2):
        # Compute the context vectors for both inputs
        context1, _ = self.attention(input1, input2, input2)

        # Compute the outputs for both inputs
        output1 = self.norm (input1 + self.dropout(context1))

        return output1

# Define the classification model
class ClassificationModel(nn.Module):
    def __init__(self,  modality1_len , modality2_len , output_dim=3, dropout=0.0):
        super(ClassificationModel, self).__init__()

#         print(modality1_len)
#         print(modality2_len)

        #modality1
        self.cross_attention_modality1_user = CrossAttention(input_dim = modality1_len, num_heads=4, dropout=0.0)
        self.cross_attention_user_modality1 = CrossAttention(input_dim = modality1_len, num_heads=4, dropout=0.0)

        #modality2
        self.cross_attention_modality2_user = CrossAttention(input_dim = modality2_len, num_heads=4, dropout=0.0)
        self.cross_attention_user_modality2 = CrossAttention(input_dim = modality2_len, num_heads=4, dropout=0.0)

        #All
        self.cross_attention_concat = CrossAttention(input_dim = (modality1_len + modality2_len)*2, num_heads=4, dropout=0.0)

        #Feed Forward layers
        self.feedForward = nn.Sequential(
                nn.Linear((modality1_len+modality2_len)*2, (modality1_len+modality2_len)),
                nn.ReLU(),
                nn.Linear((modality1_len+modality2_len), (modality1_len+modality2_len)*2)
            )

        # Regression layers
        self.classify1 = nn.Linear((modality1_len+modality2_len)*2, (modality1_len+modality2_len))
        self.classify2 = nn.Linear((modality1_len+modality2_len), (modality1_len+modality2_len)//2)
        self.classify3 = nn.Linear((modality1_len+modality2_len)//2, output_dim)

        # norm layer
        self.norm = nn.LayerNorm((modality1_len+modality2_len)*2)

    def forward(self,  modality1_embed , UserModality1_embed , modality2_embed , UserModality2_embed):

        # Compute the outputs of the cross attention model

        #modality1
        output1 = self.cross_attention_modality1_user (modality1_embed, UserModality1_embed)
        output2 = self.cross_attention_user_modality1(UserModality1_embed, modality1_embed)

        output_concat1 = torch.cat((output1,output2), 2)


        #modality2
        output3 = self.cross_attention_modality2_user(modality2_embed, UserModality2_embed)
        output4 = self.cross_attention_user_modality2(UserModality2_embed, modality2_embed)
        output_concat2 = torch.cat((output3,output4), 2)

        #All-attention
        output_concat = torch.cat((output_concat1,output_concat2), 2)
        final_output = self.cross_attention_concat(output_concat, output_concat)

        feed_out = self.feedForward(final_output)
        output = self.norm(final_output+feed_out)


        output = self.classify1(output)
        output = nn.functional.relu(output)
        output = self.classify2(output)
        output = nn.functional.relu(output)
        output = self.classify3(output).squeeze(1)

        return output

        #Here Output is of Classification type

### Train Function

In [ ]:
def train(model, dataloader, optimizer, criterion, train_data, device):
    print('Training')
    model.train()
    counter = 0
    train_running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        counter += 1
        movie_id, x_modality1, x_UserModality1, x_modality2, x_UserModality2, target = data

        x_modality1 =  x_modality1.to(device)
        x_UserModality1 = x_UserModality1.to(device)
        x_modality2 =  x_modality2.to(device)
        x_UserModality2 = x_UserModality2.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        outputs = model(x_modality1, x_UserModality1, x_modality2, x_UserModality2)
#         print(outputs.shape)
#         print(target.shape)
        loss = criterion(outputs, target)
        train_running_loss += loss.item()

        # backpropagation
        loss.backward()

        # update optimizer parameters
        optimizer.step()

    train_loss = train_running_loss / counter
    return train_loss

### Test-Function

In [ ]:
def validate(model, dataloader, criterion, val_data, device):
    print('Validating')
    model.eval()
    counter = 0
    val_running_loss = 0.0
    running_accuracy = 0.0

    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):
            counter += 1
            movie_id, x_modality1, x_UserModality1, x_modality2, x_UserModality2, target = data

            x_modality1 =  x_modality1.to(device)
            x_UserModality1 = x_UserModality1.to(device)
            x_modality2 =  x_modality2.to(device)
            x_UserModality2 = x_UserModality2.to(device)
            target = target.to(device)

            outputs = model(x_modality1, x_UserModality1, x_modality2, x_UserModality2)
            loss = criterion(outputs, target)
            val_running_loss += loss.item()

            #Calculating Accuracy
            accuracy = (torch.argmax(outputs, 1) == torch.argmax(target, 1)).float().mean()

        val_loss = val_running_loss / counter
        return val_loss , accuracy , target , outputs

In [ ]:
device = torch.device("cuda:0")
print(device)


In [ ]:
#Name Of file to save model
save_path = "best_model_ALL.pth"

# learning parameters
from torch.optim.lr_scheduler import StepLR
early_stop_thresh = 15
epochs = 150

print("Model Transfered")


#Train - Test Split
X = []
Y = []
for sample in data_file_final:
    X.append(sample)
    Y.append(int(sample[3][0]))

import collections
# using Counter to find frequency of elements
frequency = collections.Counter(Y)
print(dict(frequency))


sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

for i, (train_index, test_index) in enumerate(sss.split(X, Y)):
    train_data_X = []
    test_data = []

    for trn_indx in train_index:
        train_data_X.append(X[trn_indx])

    for tst_indx in test_index:
        test_data.append(X[tst_indx])


#Train - Val Split
X = []
Y = []
for sample in train_data_X:
    X.append(sample)
    Y.append(int(sample[3][0]))

sss1 = StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=0)


#Running for 5 Splits
for i, (train_index, val_index) in enumerate(sss1.split(X, Y)):
    best_loss = 1000
    best_epoch = -1
    valid_epoch_loss = -1
    lr = 0.00001


    train_data = []
    val_data = []
    valid_acc = []

    for trn_indx in train_index:
        train_data.append(X[trn_indx])

    for val_indx in val_index:
        val_data.append(X[val_indx])

    #Definig Loaders
    train_set = customDataset(train_data, modality1, user_modality1, modality2, user_modality2)
    val_set = customDataset(val_data, modality1, user_modality1, modality2, user_modality2)
    test_set = customDataset(test_data, modality1, user_modality1, modality2, user_modality2)

    print("Data in Train set:" , len(train_set))
    print("Data in Val set:" , len(val_set))
    print("Data in Test set:" , len(test_set))

    train_loader = torch.utils.data.DataLoader(dataset = train_set , batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset = val_set , batch_size=len(val_set))

    #Calculating the Dimesion of embeddings
    modality1_len = len(next(iter(train_set))[1][0])
    modality2_len = len(next(iter(train_set))[3][0])

    #Trainig Loop
    model = ClassificationModel(modality1_len , modality2_len)
    model = model.to(device)
    optimizer = optim.RMSprop(model.parameters(), lr=lr)
    scheduler = StepLR(optimizer,
                   step_size = 50, # Period of learning rate decay
                   gamma = 0.1) # Multiplicative factor of learning rate decay
    criterion = nn.CrossEntropyLoss()  # mean square error

    #Contains train_loss and valid_loss for the complete epochs
    train_loss = []
    valid_loss = []
    valid_acc = []

    for epoch in range(epochs):
        scheduler.step()
        print(f"Epoch {epoch+1} of {epochs}" , "Learnig Rate: " ,scheduler.get_last_lr())

        #CALLING TRAINING FUNCTION
        train_epoch_loss = train(
            model, train_loader, optimizer, criterion, train_set, device
        )

         #CALLING VALIDATION FUNCTION
        valid_epoch_loss , accuracy_score_epoch , target , outputs = validate(
            model, val_loader, criterion, val_set, device
        )

        train_loss.append(train_epoch_loss)
        valid_loss.append(valid_epoch_loss)
        valid_acc.append(accuracy_score_epoch.item())

        print(f"Train Loss: {train_epoch_loss:.4f}")
        print(f'Val Loss: {valid_epoch_loss:.4f}')
        print(f'Val Acc: {accuracy_score_epoch:.4f}')

        if valid_epoch_loss < best_loss:
            best_loss = valid_epoch_loss
            best_epoch = epoch
            checkpoint(model, save_path)

        elif epoch - best_epoch > early_stop_thresh:
            print("Early stopped training at epoch %d" % epoch)
            break  # terminate the training loop

    #Testing The model
    resume(save_path)
    model.to(device)
    test_loader = torch.utils.data.DataLoader(dataset = test_set , batch_size=len(test_set))
    print(len(test_set))

    test_epoch_loss , Testaccuracy_score_epoch, target , outputs = validate(
            model, test_loader, criterion, test_set, device
        )

    y_true = target.cpu().data.numpy().argmax(axis=-1)
    y_pred = outputs.cpu().data.numpy().argmax(axis=-1)

    cal_accuracy(y_true, y_pred)
    print(f'Test Loss: {test_epoch_loss:.4f}')
    print(f'Test Acc: {Testaccuracy_score_epoch:.4f}')
